## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-29-13-00-00 +0000,missionlocal,Where are my shoes?,https://missionlocal.org/2025/09/sf-photos-whe...
1,2025-09-29-12-55-00 +0000,wsj,Trump’s Gaza Cease-Fire Plan Faces Obstacles a...,https://www.wsj.com/world/middle-east/trumps-g...
2,2025-09-29-12-52-35 +0000,nyt,"Trump Raises Possibility of Inquiry Into Wray,...",https://www.nytimes.com/2025/09/28/us/politics...
3,2025-09-29-12-50-57 +0000,nyt,How Adams Lost His Coalition and Became an Als...,https://www.nytimes.com/2025/09/29/nyregion/ad...
4,2025-09-29-12-48-38 +0000,nyt,Still Cashing In on the ‘Field of Dreams’,https://www.nytimes.com/2025/09/28/business/fi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2297/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,32
210,new,13
90,church,13
138,shutdown,12
91,shooting,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
177,2025-09-28-20-43-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...,106
93,2025-09-29-09-00-00 +0000,wsj,Congressional leaders will head to the White H...,https://www.wsj.com/politics/policy/government...,83
16,2025-09-29-12-13-00 +0000,nypost,Trump blasts Michigan church shooting as ‘atta...,https://nypost.com/2025/09/29/us-news/trump-bl...,82
133,2025-09-29-01-58-54 +0000,nypost,President Trump shows off new White House blin...,https://nypost.com/2025/09/28/us-news/presiden...,77
145,2025-09-28-23-53-00 +0000,wsj,"Eric Adams Ends Re-Election Bid, Reshaping Rac...",https://www.wsj.com/us-news/new-york-city-mayo...,66


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
177,106,2025-09-28-20-43-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...
145,66,2025-09-28-23-53-00 +0000,wsj,"Eric Adams Ends Re-Election Bid, Reshaping Rac...",https://www.wsj.com/us-news/new-york-city-mayo...
204,56,2025-09-28-16-54-04 +0000,cbc,"4 dead, 8 wounded in shooting and fire at Mich...",https://www.cbc.ca/news/world/multiple-people-...
93,50,2025-09-29-09-00-00 +0000,wsj,Congressional leaders will head to the White H...,https://www.wsj.com/politics/policy/government...
8,34,2025-09-29-12-30-52 +0000,nypost,Netanyahu teases immunity for Hamas in propose...,https://nypost.com/2025/09/29/world-news/netan...
72,28,2025-09-29-09-30-00 +0000,wsj,"The Pentagon, alarmed at the low weapons stock...",https://www.wsj.com/politics/national-security...
17,26,2025-09-29-12-05-00 +0000,wsj,Moldova’s Pro-Europe Party Claims Victory Over...,https://www.wsj.com/world/europe/moldovas-pro-...
92,25,2025-09-29-09-00-00 +0000,wsj,"Rahm Emanuel, former Democratic congressman, W...",https://www.wsj.com/politics/policy/rahm-emanu...
68,24,2025-09-29-09-45-12 +0000,bbc,Denmark bans all civilian drone flights ahead ...,https://www.bbc.com/news/articles/c708dzqvz1vo...
213,24,2025-09-28-15-48-56 +0000,nypost,James Comey’s ex-No. 2 gives mealy-mouthed ans...,https://nypost.com/2025/09/28/us-news/comeys-e...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
